In [ ]:
with open("/content/drive/MyDrive/genai_lora_dataset.jsonl") as f:
    for _ in range(5):
        print(f.readline())


{"instruction": "[INST] <<SYS>>You are a lead conversion prediction model. Analyze lead profiles and predict conversion as 0 (will not convert) or 1 (will convert).<</SYS>>\n\nAnalyze this lead profile:\n\nSource (marketing channel and acquisition method):\n- Lead Source: Chat/Live Support\n- Lead Origin: API\n\nEngagement (website interaction metrics):\n- Total Visits: None\n- Time on Website: 0.0 seconds\n- Page Views per Visit: 0.0\n\nLocation (geographic data):\n- Country: Unknown\n- City: Unknown\n\nBackground (professional profile):\n- Occupation: Unemployed\n- Specialization: Unknown\n- Profile: Unknown\n\nPreferences (intent signals):\n- Course Priority: Carrer Prospects\n- Interview Guide: No\n- Do Not Email: No\n\nPredict conversion (0 or 1): [/INST]", "response": "0"}

{"instruction": "[INST] <<SYS>>You are a lead conversion prediction model. Analyze lead profiles and predict conversion as 0 (will not convert) or 1 (will convert).<</SYS>>\n\nAnalyze this lead profile:\n\nSou

In [ ]:
import json

input_path = "/content/drive/MyDrive/genai_lora_dataset.jsonl"
output_path = "/content/drive/MyDrive/genai_lora_dataset_clean.jsonl"

with open(input_path, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        obj = json.loads(line)

        instr = obj["instruction"]

        # Remove LLaMA chat wrappers
        instr = instr.replace("[INST]", "")
        instr = instr.replace("[/INST]", "")
        instr = instr.replace("<<SYS>>", "")
        instr = instr.replace("<</SYS>>", "")

        instr = instr.strip()

        fout.write(json.dumps({
            "instruction": instr,
            "response": obj["response"]
        }) + "\n")

print("Cleaned dataset saved.")


Cleaned dataset saved.


In [ ]:
with open("/content/drive/MyDrive/genai_lora_dataset_clean.jsonl") as f:
    for _ in range(3):
        print(f.readline())


{"instruction": "You are a lead conversion prediction model. Analyze lead profiles and predict conversion as 0 (will not convert) or 1 (will convert).\n\nAnalyze this lead profile:\n\nSource (marketing channel and acquisition method):\n- Lead Source: Chat/Live Support\n- Lead Origin: API\n\nEngagement (website interaction metrics):\n- Total Visits: None\n- Time on Website: 0.0 seconds\n- Page Views per Visit: 0.0\n\nLocation (geographic data):\n- Country: Unknown\n- City: Unknown\n\nBackground (professional profile):\n- Occupation: Unemployed\n- Specialization: Unknown\n- Profile: Unknown\n\nPreferences (intent signals):\n- Course Priority: Carrer Prospects\n- Interview Guide: No\n- Do Not Email: No\n\nPredict conversion (0 or 1):", "response": "0"}

{"instruction": "You are a lead conversion prediction model. Analyze lead profiles and predict conversion as 0 (will not convert) or 1 (will convert).\n\nAnalyze this lead profile:\n\nSource (marketing channel and acquisition method):\n- L

In [ ]:
import json

input_path = "/content/drive/MyDrive/genai_lora_dataset_clean.jsonl"
output_path = "/content/drive/MyDrive/genai_lora_dataset_final.jsonl"

with open(input_path, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        obj = json.loads(line)

        instr = obj["instruction"]

        # Remove generic role instruction
        instr = instr.replace(
            "You are a lead conversion prediction model. Analyze lead profiles and predict conversion as 0 (will not convert) or 1 (will convert).",
            ""
        ).strip()

        fout.write(json.dumps({
            "instruction": instr,
            "response": obj["response"]
        }) + "\n")

print("Final GENAI dataset ready.")


Final GENAI dataset ready.


In [ ]:
with open("/content/drive/MyDrive/genai_lora_dataset_final.jsonl") as f:
    for _ in range(3):
        print(f.readline())


{"instruction": "Analyze this lead profile:\n\nSource (marketing channel and acquisition method):\n- Lead Source: Chat/Live Support\n- Lead Origin: API\n\nEngagement (website interaction metrics):\n- Total Visits: None\n- Time on Website: 0.0 seconds\n- Page Views per Visit: 0.0\n\nLocation (geographic data):\n- Country: Unknown\n- City: Unknown\n\nBackground (professional profile):\n- Occupation: Unemployed\n- Specialization: Unknown\n- Profile: Unknown\n\nPreferences (intent signals):\n- Course Priority: Carrer Prospects\n- Interview Guide: No\n- Do Not Email: No\n\nPredict conversion (0 or 1):", "response": "0"}

{"instruction": "Analyze this lead profile:\n\nSource (marketing channel and acquisition method):\n- Lead Source: Organic Search\n- Lead Origin: API\n\nEngagement (website interaction metrics):\n- Total Visits: Low\n- Time on Website: 674.0 seconds\n- Page Views per Visit: 2.5\n\nLocation (geographic data):\n- Country: India\n- City: Unknown\n\nBackground (professional prof

In [1]:
!pip install -q transformers accelerate peft bitsandbytes datasets trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 29.4 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [3]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
from peft import LoraConfig, get_peft_model

In [6]:
lora_config = LoraConfig(
    r=8,                    # rank (small = efficient)
    lora_alpha=16,          # scaling factor
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [7]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [8]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/genai_lora_dataset_final.jsonl"
)


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)


In [10]:
def format_example(example):
    text = example["instruction"] + "\n" + example["response"]
    return {"text": text}


In [11]:
dataset = dataset.map(format_example)


Map:   0%|          | 0/7421 [00:00<?, ? examples/s]

Map:   0%|          | 0/1856 [00:00<?, ? examples/s]

In [12]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

    labels = tokens["input_ids"].copy()

    labels = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in labels
    ]

    tokens["labels"] = labels
    return tokens


In [13]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/7421 [00:00<?, ? examples/s]

Map:   0%|          | 0/1856 [00:00<?, ? examples/s]

In [14]:
from transformers import TrainingArguments

In [15]:
training_args = TrainingArguments(
    output_dir="./lora-lead-conversion",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    eval_strategy="epoch", # Changed from evaluation_strategy
    save_strategy="epoch",
    logging_steps=50,
    report_to="none"
)

In [16]:
from transformers import Trainer

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)


/tmp/ipython-input-1871281149.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Epoch,Training Loss,Validation Loss
1,0.075200,0.078095
2,0.075300,0.075025
3,0.073800,0.073357


TrainOutput(global_step=1392, training_loss=0.11537390399253232, metrics={'train_runtime': 4508.0637, 'train_samples_per_second': 4.938, 'train_steps_per_second': 0.309, 'total_flos': 7.082934143444582e+16, 'train_loss': 0.11537390399253232, 'epoch': 3.0})

In [19]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [20]:
import torch
torch.set_grad_enabled(False)


torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [21]:
def predict_label(instruction, max_new_tokens=2):
    inputs = tokenizer(
        instruction,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,        # deterministic
        temperature=0.0,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract last character safely
    if decoded.strip().endswith("1"):
        return 1
    else:
        return 0


In [22]:
y_true = []
y_pred = []

test_data = dataset["test"]

for sample in test_data:
    y_true.append(int(sample["response"]))
    y_pred.append(predict_label(sample["instruction"]))


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [23]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)


In [24]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)


In [25]:
print("GENAI–LoRA Model Evaluation")
print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-score  : {f1:.4f}")
print(f"ROC-AUC   : {roc_auc:.4f}")
print("Confusion Matrix:\n", cm)


GENAI–LoRA Model Evaluation
Accuracy  : 0.8028
Precision : 0.7031
Recall    : 0.7811
F1-score  : 0.7401
ROC-AUC   : 0.7980
Confusion Matrix:
 [[969 220]
 [146 521]]
